# Team 10 Final Project - Santa's Workshop

## Team Member: Bianca Gunawan, Jacky Lin, Kevin Ko, Sally Lee

In [1]:
import Pkg
Pkg.add("HiGHS")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using JuMP, HiGHS, CSV, DataFrames

file_path = "/Users/sally/Desktop/ADSP 32013 Optimization/Final Project/family_data.csv"
df = CSV.read(file_path, DataFrame)

df

Row,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,52,38,12,82,33,75,64,76,10,28,4
2,1,26,4,82,5,11,47,38,6,66,61,4
3,2,100,54,25,12,27,82,10,89,80,33,3
4,3,2,95,1,96,32,6,40,31,9,59,2
5,4,53,1,47,93,26,3,46,16,42,39,4
6,5,32,59,12,3,60,26,35,50,5,2,4
7,6,88,4,1,3,91,32,39,57,28,99,2
8,7,25,11,52,48,10,17,88,50,95,66,5
9,8,18,60,1,12,89,33,16,10,53,67,4


# Start testing with 10 families and 10 days (10, 10)

In [3]:
#Sample size (10,10)

n, m = 10, 10

model = Model(HiGHS.Optimizer)

@variable(model, Family_Attendance[1:n, 1:m], Bin)

function num_people(i)
    return df[i, "n_people"]
end

function cost_per_family(i,j)
    if j == df[i, "choice_0"]
        c = 0
    elseif j == df[i, "choice_1"]
        c = 50
    elseif j == df[i, "choice_2"]
        c = 50 + 9 * num_people(i)
    elseif j == df[i, "choice_3"]
        c = 100 + 9 * num_people(i)
    elseif j == df[i, "choice_4"]
        c = 200 + 9 * num_people(i)
    elseif j == df[i, "choice_5"]
        c = 200 + 18 * num_people(i)
    elseif j == df[i, "choice_6"]
        c = 300 + 18 * num_people(i)
    elseif j == df[i, "choice_7"]
        c = 300 + 36 * num_people(i)
    elseif j == df[i, "choice_8"]
        c = 400 + 36 * num_people(i)
    elseif j == df[i, "choice_9"]
        c = 699 + 36 * num_people(i)
    else
        c = 898 + 36 * num_people(i)
    end
    return c
end

#row
for i=1:n
    @constraint(model, sum(Family_Attendance[i,:])==1)
end

#column
for j=1:m
    @constraint(model, 0 <= sum(Family_Attendance[:, j] .* df[1:n, "n_people"]) <= 10)
end

@objective(model, Min, sum(sum(cost_per_family(i, j) * Family_Attendance[i, j] for j=1:m) for i=1:n))

JuMP.optimize!(model)

term_status = JuMP.termination_status(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
20 rows, 100 cols, 200 nonzeros
20 rows, 100 cols, 200 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   20 rows
   100 cols (100 binary, 0 integer, 0 implied int., 0 continuous)
   200 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   2150.285714     3191              32.61%        0      0      0        13     0.0s
 L       0       0         0   0.00%   2725.5          3163              13.83%       11      1     10        16     0.0s

45.0% inactive integer columns, restarting
Model after restart has 16 rows

OPTIMAL::TerminationStatusCode = 1

In [4]:
# Check if the optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Print the optimal total cost
    println("Optimal Total Cost: ", objective_value(model))

    # Print number of people visiting each day (z)
    println("Number of people visiting each day:")
    for j in 1:m
        println("Day $j: ", sum(value(Family_Attendance[i, j]) * num_people(i) for i in 1:n), " people")
    end
else
    println("No optimal solution found.")
end

Optimal Total Cost: 3163.0
Number of people visiting each day:
Day 1: 8.0 people
Day 2: 2.0 people
Day 3: 4.0 people
Day 4: 6.0 people
Day 5: 4.0 people
Day 6: 0.0 people
Day 7: 7.0 people
Day 8: 0.0 people
Day 9: 0.0 people
Day 10: 8.0 people


# Start testing with 100 families and 10 days (100, 10)

In [5]:
#Sample size (100,10)

n, m = 100, 10

model = Model(HiGHS.Optimizer)

@variable(model, Family_Attendance[1:n, 1:m], Bin)

function num_people(i)
    return df[i, "n_people"]
end

function cost_per_family(i,j)
    if j == df[i, "choice_0"]
        c = 0
    elseif j == df[i, "choice_1"]
        c = 50
    elseif j == df[i, "choice_2"]
        c = 50 + 9 * num_people(i)
    elseif j == df[i, "choice_3"]
        c = 100 + 9 * num_people(i)
    elseif j == df[i, "choice_4"]
        c = 200 + 9 * num_people(i)
    elseif j == df[i, "choice_5"]
        c = 200 + 18 * num_people(i)
    elseif j == df[i, "choice_6"]
        c = 300 + 18 * num_people(i)
    elseif j == df[i, "choice_7"]
        c = 300 + 36 * num_people(i)
    elseif j == df[i, "choice_8"]
        c = 400 + 36 * num_people(i)
    elseif j == df[i, "choice_9"]
        c = 699 + 36 * num_people(i)
    else
        c = 898 + 36 * num_people(i)
    end
    return c
end

#row
for i=1:n
    @constraint(model, sum(Family_Attendance[i,:])==1)
end

#column
for j=1:m
    @constraint(model, 0 <= sum(Family_Attendance[:, j] .* df[1:n, "n_people"]) <= 50)
end

@objective(model, Min, sum(sum(cost_per_family(i, j) * Family_Attendance[i, j] for j=1:m) for i=1:n))

JuMP.optimize!(model)

term_status = JuMP.termination_status(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
110 rows, 1000 cols, 2000 nonzeros
110 rows, 1000 cols, 2000 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   110 rows
   1000 cols (1000 binary, 0 integer, 0 implied int., 0 continuous)
   2000 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   33645.285714    35882              6.23%        0      0      0       129     0.0s
 L       0       0         0   0.00%   33848           33848              0.00%      112     14     11       154     0.0s

Solving report
  Status            Optimal
  Primal bound      3

OPTIMAL::TerminationStatusCode = 1

In [6]:
# Check if the optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Print the optimal total cost
    println("Optimal Total Cost: ", objective_value(model))

    # Print number of people visiting each day (z)
    println("Number of people visiting each day:")
    for j in 1:m
        println("Day $j: ", sum(value(Family_Attendance[i, j]) * num_people(i) for i in 1:n), " people")
    end
else
    println("No optimal solution found.")
end

Optimal Total Cost: 33848.0
Number of people visiting each day:
Day 1: 50.0 people
Day 2: 45.0 people
Day 3: 50.0 people
Day 4: 49.0 people
Day 5: 50.0 people
Day 6: 12.0 people
Day 7: 25.0 people
Day 8: 44.0 people
Day 9: 22.0 people
Day 10: 50.0 people


# Full Scale of 5000 families and 100 days (5000, 100)

In [ ]:
n, m = 5000, 100

min_cost = 10000
max_cost = 60000

model = Model(HiGHS.Optimizer)

@variable(model, Family_Attendance[1:n, 1:m], Bin)

function num_people(i)
    return df[i, "n_people"]
end

function cost_per_family(i,j)
    if j == df[i, "choice_0"]
        c = 0
    elseif j == df[i, "choice_1"]
        c = 50
    elseif j == df[i, "choice_2"]
        c = 50 + 9 * num_people(i)
    elseif j == df[i, "choice_3"]
        c = 100 + 9 * num_people(i)
    elseif j == df[i, "choice_4"]
        c = 200 + 9 * num_people(i)
    elseif j == df[i, "choice_5"]
        c = 200 + 18 * num_people(i)
    elseif j == df[i, "choice_6"]
        c = 300 + 18 * num_people(i)
    elseif j == df[i, "choice_7"]
        c = 300 + 36 * num_people(i)
    elseif j == df[i, "choice_8"]
        c = 400 + 36 * num_people(i)
    elseif j == df[i, "choice_9"]
        c = 699 + 36 * num_people(i)
    else
        c = 898 + 36 * num_people(i)
    end
    return c
end

@constraint(model, sum(sum(cost_per_family(i, j) * Family_Attendance[i, j] for j=1:m) for i=1:n) >= min_cost)
@constraint(model, sum(sum(cost_per_family(i, j) * Family_Attendance[i, j] for j=1:m) for i=1:n) <= max_cost)

#row
for i=1:n
    @constraint(model, sum(Family_Attendance[i,:])==1)
end

#column
for j=1:m
    @constraint(model, 125 <= sum(Family_Attendance[:, j] .* df[1:n, "n_people"]) <= 300)
end

@objective(model, Min, sum(sum(cost_per_family(i, j) * Family_Attendance[i, j] for j=1:m) for i=1:n))

JuMP.optimize!(model)

term_status = JuMP.termination_status(model)

In [ ]:
# Check if the optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Print the optimal total cost
    println("Optimal Total Cost: ", objective_value(model))

    # Print number of people visiting each day (z)
    println("Number of people visiting each day:")
    for j in 1:m
        println("Day $j: ", sum(value(Family_Attendance[i, j]) * num_people(i) for i in 1:n), " people")
    end
else
    println("No optimal solution found.")
end

In [ ]:
result = DataFrame()

family = []
days = []


for i in 1:5000
    for j in 1:100
        if value(Family_Attendance[i, j]) == 1
            push!(family, i-1)
            push!(days, j)
        end
    end
end

result.family_id = family
result.days = days

result